In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm


# Defining teams and codes

In [2]:
dict_longteam_to_shortteam = {'atlanta-hawks':'ATL','boston-celtics':'BOS','brooklyn-nets':'BK','charlotte-hornets':'CHA',
                              'chicago-bulls':'CHI','cleveland-cavaliers':'CLE','dallas-mavericks':'DAL','denver-nuggets':'DEN',
                              'detroit-pistons':'DET','golden-state-warriors':'GS','houston-rockets':'HOU','indiana-pacers':'IND',
                              'los-angeles-clippers':'LAC','los-angeles-lakers':'LAL','memphis-grizzlies':'MEM','miami-heat':'MIA',
                              'milwaukee-bucks':'MIL','minnesota-timberwolves':'MIN','new-orleans-pelicans':'NO','new-york-knicks':'NY',
                              'oklahoma-city-thunder':'OKC','orlando-magic':'ORL','philadelphia-76ers':'PHI','phoenix-suns':'PHO',
                              'portland-trail-blazers':'POR','sacramento-kings':'SAC','san-antonio-spurs':'SA','toronto-raptors':'TOR',
                              'utah-jazz':'UTA','washington-wizards':'WAS',
                             }

dict_shortteam_to_codeteam = {'ATL':1001,'BOS':1002,'BK':1003,'CHA':1004,'CHI':1005,'CLE':1006,'DAL':1007,
                              'DEN':1008,'DET':1009,'GS':1010,'HOU':1011,'IND':1012,'LAC':1013,'LAL':1014,
                              'MEM':1015,'MIA':1016,'MIL':1017,'MIN':1018,'NO':1019,'NY':1020,'OKC':1021,
                              'ORL':1022,'PHI':1023,'PHO':1024,'POR':1025,'SAC':1026,'SA':1027,'TOR':1028,
                              'UTA':1029,'WAS':1030,
                             }

dict_codeteam_to_shortteam = {1001:'ATL',1002:'BOS',1003:'BK',1004:'CHA',1005:'CHI',1006:'CLE',1007:'DAL',
                              1008:'DEN',1009:'DET',1010:'GS',1011:'HOU',1012:'IND',1013:'LAC',1014:'LAL',
                              1015:'MEM',1016:'MIA',1017:'MIL',1018:'MIN',1019:'NO',1020:'NY',1021:'OKC',
                              1022:'ORL',1023:'PHI',1024:'PHO',1025:'POR',1026:'SAC',1027:'SA',1028:'TOR',
                              1029:'UTA',1030:'WAS',
                             }


# Reading historical data

In [3]:
teams = ['new-york-knicks','brooklyn-nets','boston-celtics','philadelphia-76ers','toronto-raptors'
        ,'chicago-bulls','cleveland-cavaliers','detroit-pistons','indiana-pacers','milwaukee-bucks'
        ,'atlanta-hawks','charlotte-hornets','miami-heat','orlando-magic','washington-wizards'
        ,'denver-nuggets','minnesota-timberwolves','oklahoma-city-thunder','portland-trail-blazers','utah-jazz'
        ,'golden-state-warriors','los-angeles-clippers','los-angeles-lakers','phoenix-suns','sacramento-kings'
        ,'dallas-mavericks','houston-rockets','memphis-grizzlies','new-orleans-pelicans','san-antonio-spurs']
# teams = [utah-jazz']
# seasons = ['2006-2007','2007-2008','2008-2009','2009-2010','2010-2011','2011-2012'
#            ,'2012-2013','2013-2014','2014-2015','2015-2016','2016-2017','2017-2018','2018-2019','2019-2020',]
seasons = ['2011-2012']
my_columns = ['Season','Team','Date', 'VS', 'Score', 'ATS', 'O/U', 'filled-1', 'filled-2']
df = pd.DataFrame(columns = my_columns)
for team in tqdm(teams):
    for season in seasons: 
        r = pd.read_html(f'https://www.covers.com/sport/basketball/nba/teams/main/{team}/{season}', match='Regular Season')
        r[0].insert(0,'Season',int)
        r[0].insert(1,'Team',int)
        r[0]['Season'] = season
        r[0]['Team'] = team
        r[0].columns = my_columns
        df = df.append(r[0])

df.drop(['filled-1', 'filled-2'], axis=1, inplace=True)

# Extracting the year to attach to the field Date
year_season_star =df['Season'].str.split('-').str[0]
year_season_end =df['Season'].str.split('-').str[1]

# Attaching the year to the field Date
df['Date'] = list(np.select([df['Date'].str[:3].isin(['Oct','Nov','Dec']), 
                             ~df['Date'].str[:3].isin(['Oct','Nov','Dec'])] , 
                            [df['Date']+'-'+year_season_star, df['Date']+'-'+year_season_end], default= 0))

df['Date'] = pd.to_datetime(df['Date'],)
df = df.iloc[::-1]
df['GameCounter'] = df.groupby(['Team']).cumcount()+1
df.set_index('GameCounter', inplace=True)


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:23<00:00,  1.25it/s]


In [4]:
# Updating Nets and NJ to BK; CHAR to CHA
df['VS'] = df['VS'].str.replace('Nets', 'BK')
df['VS'] = df['VS'].str.replace('NJ', 'BK')
df['VS'] = df['VS'].str.replace('CHAR', 'CHA')

# Changing long name in team for the short name; Creating new column codeTeam and assigning the code
df['Team'] = df['Team'].replace(to_replace=dict_longteam_to_shortteam, value=None)
df['codeTeam'] = df['Team'].replace(to_replace=dict_shortteam_to_codeteam, value=None)

# Creating new column codeVS and assigning the code
df['codeVS'] = df['VS']
df['codeVS'] = df['codeVS'].str.replace('@','')
df['codeVS'] = df['codeVS'].str.replace('NEUTRAL','')
df['codeVS'] = df['codeVS'].str.replace(' ','')
df['codeVS'] = df['codeVS'].replace(to_replace=dict_shortteam_to_codeteam, value=None)

# Saving the data in csv

In [8]:
df.to_csv('Sports_NBA_History.csv')